In [1]:
!python3 test.py --dataset_mode ade20k --name oasis_ade20k_pretrained --dataroot data/ADEChallengeData2016/ --batch 8


----------------- Options ---------------
                EMA_decay: 0.9999                        
               batch_size: 8                             	[default: 1]
               channels_G: 64                            
          checkpoints_dir: ./checkpoints                 
                ckpt_iter: best                          
                 dataroot: data/ADEChallengeData2016/    	[default: ./datasets/cityscapes/]
             dataset_mode: ade20k                        	[default: coco]
                  gpu_ids: 0                             
                     name: oasis_ade20k_pretrained       	[default: label2coco]
               no_3dnoise: False                         
                   no_EMA: False                         
                  no_flip: False                         
         no_spectral_norm: False                         
           num_res_blocks: 6                             
              num_workers: 0                             
   

In [3]:
import argparse
import config
import utils.utils as utils

import models.models as models
import dataloaders.dataloaders as dataloaders
import utils.utils as utils
import config

parser = argparse.ArgumentParser()
type(parser)
parser = config.add_all_arguments(parser, train=False)

In [4]:

opt = parser.parse_args("--dataset_mode ade20k --name oasis_ade20k_pretrained --dataroot data/ADEChallengeData2016/ --batch 4 --num_workers 4".split()) 
opt.phase = "test"
opt.continue_train = False
opt.add_vgg_loss = False
utils.fix_seed(opt.seed)
# --ckpt_iter='best' --contain_dontcare_label=True --crop_size=256 --dataroot='data/ADEChallengeData2016/', \
# dataset_mode='ade20k', gpu_ids='0', label_nc=150, load_size=256, name='oasis_ade20k_pretrained', no_3dnoise=False, \
# no_EMA=False, no_flip=False, no_spectral_norm=False, num_res_blocks=6, num_workers=0, \
# param_free_norm='syncbatch', phase='test', results_dir='./results/', seed=42, semantic_nc=151, spade_ks=3, z_dim=64")

opt = "EMA_decay=0.9999, aspect_ratio=1.0, batch_size=8, cache_filelist_read=False, cache_filelist_write=False, channels_G=64, checkpoints_dir='./checkpoints', \
ckpt_iter='best', contain_dontcare_label=True, crop_size=256, dataroot='data/ADEChallengeData2016/', \
dataset_mode='ade20k', gpu_ids='0', label_nc=150, load_size=256, name='oasis_ade20k_pretrained', no_3dnoise=False, \
no_EMA=False, no_flip=False, no_spectral_norm=False, num_res_blocks=6, num_workers=0, \
param_free_norm='syncbatch', phase='test', results_dir='./results/', seed=42, semantic_nc=151, spade_ks=3, z_dim=64"

In [5]:


#--- read options ---#
# opt = config.read_arguments(train=False)

#--- create dataloader ---#
_, dataloader_val = dataloaders.get_dataloaders(opt)

#--- create utils ---#
image_saver = utils.results_saver(opt)
 
#--- create models ---#
model = models.OASIS_model(opt).cuda()
#model = models.put_on_multi_gpus(model, opt)
model.eval()
#--- iterate over validation set ---#
for i, data_i in enumerate(dataloader_val):
    _, label = models.preprocess_input(opt, data_i)
    generated = model(None, label, "generate", None)
    image_saver(label, generated, data_i["name"])


Created Ade20kDataset, size train: 2000, size val: 2000
Created OASIS_Generator with 74314691 parameters


/home/alexander/.local/lib/python3.8/site-packages/torchvision/transforms/functional.py:386: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/alexander/.local/lib/python3.8/site-packages/torchvision/transforms/functional.py:386: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/alexander/.local/lib/python3.8/site-packages/torchvision/transforms/functional.py:386: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/alexander/.local/lib/python3.8/site-packages/torchvision/transforms/functional.py:386: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/alexander/.local/lib/python3.8/site-packages/torch/nn/func

KeyboardInterrupt: 

In [4]:
data_i['image'][0].mean()

tensor(-0.1748, device='cuda:0')

In [6]:
data_i['name'][0]

'ADE_val_00000021.jpg'

In [126]:
import torchvision.transforms as TR
img = Image.open("data/ADEChallengeData2016/images/validation/ADE_val_00000014.jpg").convert("RGB")
img=TR.functional.to_tensor(img)
img = TR.functional.resize(img, (256, 256), Image.BICUBIC)
img = TR.functional.normalize(img, (0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
img = img.cuda()
img.mean()

/home/alexander/.local/lib/python3.8/site-packages/torchvision/transforms/functional.py:386: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


tensor(-0.0001, device='cuda:0')

In [12]:
model.netD = models.discriminators.OASIS_Discriminator(opt).cuda()
model.netD.load_state_dict(torch.load("checkpoints/oasis_ade20k_pretrained/models/best_D.pth"))

<All keys matched successfully>

In [131]:
import torchvision.transforms as T
from PIL import Image
from PIL import Image
import torch
import numpy as np

i = 0
cls = 151
shift = 0
seed = 0
gt = new
for j in range(5):
	torch.manual_seed(seed)
	# z = torch.randn([1, 64, 256, 256], dtype=torch.float32, device="cuda:0")
	z = torch.randn(1, 64, dtype=torch.float32, device="cuda:0")
	z = z.view(1, 64, 1, 1)
	z = z.expand(1, 64, 256, 256)
	if cls==151:
		z = z + shift
	else:
		z = (z[0]+shift)*gt[i][cls] 
		z = z.unsqueeze(0)
	generated = model(None, gt[i].unsqueeze(0), "generate", None,z)

	im = utils.tens_to_lab(gt[i], 150)
	im = utils.tens_to_im(generated[0]) * 255
	image = Image.fromarray(np.uint8(im))
	image.show()
	# image.save(f"check/image{seed}.jpg")
	seed+=1

/home/alexander/.local/lib/python3.8/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/alexander/.local/lib/python3.8/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/alexander/.local/lib/python3.8/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/alexander/.local/lib/python3.8/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/alexander/.local/lib/python3.8/site-packages/torch/nn/functional.py:1794: UserWarn

In [128]:
labels = model.netD(img.unsqueeze(0))[0][1:].unsqueeze(0).cuda()
# labels[labels>0.5] = 1
# labels[labels<0.5] = 0
# labels = labels/10

In [129]:
new_labels = labels.argmax(axis=1)
new = torch.zeros([1,151,256,256]).cuda()
# new[0,new_labels] = 1

for i in range(256):
	for j in range(256):
		new[0,int(new_labels[0,i,j]),i,j] = 1
new[0,...,0,0]


tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.], device='cuda:0')

In [16]:
labels[0].shape

torch.Size([151, 256, 256])

In [20]:
new_labels.shape

torch.Size([1, 256, 256])